# 2. NLP Of Posts

In [29]:
!pip install gensim  

  Attempting uninstall: Cython
    Found existing installation: Cython 0.28.2
    Uninstalling Cython-0.28.2:
      Successfully uninstalled Cython-0.28.2


You should consider upgrading via the 'c:\dev\python.exe -m pip install --upgrade pip' command.


In [30]:
import nltk
nltk.download('stopwords')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
#import emoji
import spacy
import math
import pickle
from time import time
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn import svm

from contextlib import contextmanager
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import time
import re
import string
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc
from collections import defaultdict
import os
import psutil

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import nltk
#import contractions
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Allegra
[nltk_data]     Grunberg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\dev\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[nltk_data] Downloading package wordnet to C:\Users\Allegra
[nltk_data]     Grunberg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
#Data
wall = pd.read_csv('reddit_wsb.csv') 

Lemmatization

In [42]:
!python -m spacy download en_core_web_sm

^C


In [39]:
import en_core_web_sm

In [86]:
#lem is better then stem dont need to do both
#Consider lemmatization (reduce words such as “am”, “are”, and “is” to a common form such as “be”)
#spacy.load('en') 

import en_core_web_sm

nlp = en_core_web_sm.load() # load spacy model, can be "en_core_web_sm"

# Apply lemmatization in order to get the base form of a word
lemmatized = []

for text in wall['title']:
    doc = nlp(text)
    # Lemmatizing each token
    mytokens = [word.lemma_ if word.lemma_ != "-PrON-" else word.lower_ for word in doc]
    join_list = (' ').join(mytokens) # Join list of words into a sentence
    lemmatized.append(join_list) # Append to a list

Tokenization

In [61]:
#remove nans

wall = wall.dropna(subset=['title', 'body'])

In [62]:
# Tokenize text (split words into a list)
def split_line(text):
    # split the text
    list_words = text.split()
    return list_words

wall['title_split'] = wall['title'].apply(split_line)
wall['body_split'] = wall['body'].apply(split_line)
wall.head()  #(change comments to body etc.)

C:\dev\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\dev\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,score,id,url,comms_num,created,body,timestamp,title_split,body_split
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"[Exit, the, system]","[The, CEO, of, NASDAQ, pushed, to, halt, tradi..."
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,"[SHORT, STOCK, DOESN'T, HAVE, AN, EXPIRATION, ...","[Hedgefund, whales, are, spreading, disinfo, s..."
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,"[THIS, IS, THE, MOMENT]","[Life, isn't, fair., My, mother, always, told,..."
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,"[We, need, to, keep, this, movement, going,, w...","[I, believe, right, now, is, one, of, those, r..."
12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,"[Once, you're, done, with, GME, -, $AG, and, $...","[You, guys, are, champs., GME..., who, would, ..."


Stop Word Removal

In [71]:
# Encountering an error with nltk not being defined in order to download stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')

# Print and copy stopwords instead
#print(len(stopwords.words('english')),'\n', stopwords.words('english'))

#//input [‘abc’, ‘bcd’, ‘no’]
#//output [‘abc’,’bcd’] 
 
def remove_stopwords(my_list):
    newlist=[]
    for word in my_list:
        if word in stopwords.words('english'):
            continue
        else: 
            newlist.append(word)
    return newlist

wall['title_nostop'] = wall['title_split'].apply(remove_stopwords)
wall['body_nostop'] = wall['body_split'].apply(remove_stopwords)
wall.head()  #(change comments to body etc.)

C:\dev\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\dev\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,title,score,id,url,comms_num,created,body,timestamp,title_split,body_split,title_nostop,body_nostop
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"[Exit, the, system]","[The, CEO, of, NASDAQ, pushed, to, halt, tradi...","[Exit, system]","[The, CEO, NASDAQ, pushed, halt, trading, “to,..."
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,"[SHORT, STOCK, DOESN'T, HAVE, AN, EXPIRATION, ...","[Hedgefund, whales, are, spreading, disinfo, s...","[SHORT, STOCK, DOESN'T, HAVE, AN, EXPIRATION, ...","[Hedgefund, whales, spreading, disinfo, saying..."
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,"[THIS, IS, THE, MOMENT]","[Life, isn't, fair., My, mother, always, told,...","[THIS, IS, THE, MOMENT]","[Life, fair., My, mother, always, told, I, wou..."
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,"[We, need, to, keep, this, movement, going,, w...","[I, believe, right, now, is, one, of, those, r...","[We, need, keep, movement, going,, make, histo...","[I, believe, right, one, rare, opportunities, ..."
12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,"[Once, you're, done, with, GME, -, $AG, and, $...","[You, guys, are, champs., GME..., who, would, ...","[Once, done, GME, -, $AG, $SLV,, gentleman's, ...","[You, guys, champs., GME..., would, thought, b..."


# Sentiment Analysis!! 

In [72]:
#https://stackoverflow.com/questions/61608057/output-vader-sentiment-scores-in-columns-based-on-dataframe-rows-of-tweets
#https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
#https://medium.com/nerd-for-tech/wallstreetbets-sentiment-analysis-on-stock-prices-using-natural-language-processing-ed1e9e109a37
#https://swaggystocks.com/dashboard/wallstreetbets/realtime
#https://nbviewer.jupyter.org/github/dipanjanS/practical_nlp_workshop_gids20/blob/main/tutorials/03_NLP%20Applications/09_NLP_Applications_Text_Classification_Machine_Learning_and_Basic_DNN.ipynb

More 
#https://github.com/ProsusAI/finBERT 
#https://huggingface.co/ProsusAI/finbert
    
!pip install vaderSentiment

You should consider upgrading via the 'c:\dev\python.exe -m pip install --upgrade pip' command.


In [85]:
#vader uses normal english may n ot pick up on nuanced languages or context
#look at models -- LINKS DJ SENT !! 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

new_words = {
    'citron': -4.0,  
    'hidenburg': -4.0,        
    'moon': 4.0,
    'highs': 2.0,
    'mooning': 4.0,
    'long': 2.0,
    'short': -2.0,
    'call': 4.0,
    'calls': 4.0,    
    'put': -4.0,
    'puts': -4.0,    
    'break': 2.0,
    'tendie': 2.0,
     'tendies': 2.0,
     'town': 2.0,     
     'overvalued': -3.0,
     'undervalued': 3.0,
     'buy': 4.0,
     'sell': -4.0,
     'gone': -1.0,
     'gtfo': -1.7,
     'paper': -1.7,
     'bullish': 3.7,
     'bearish': -3}

analyzer = SentimentIntensityAnalyzer()
analyzer.lexicon.update(new_words)

wall['rating'] = wall['body'].apply(analyzer.polarity_scores)

#wall['compound'] = [analyzer.polarity_scores(x)['compound'] for x in wall['body_split']]
#wall['neg'] = [analyzer.polarity_scores(x)['neg'] for x in wall['body_split']]
#wall['neu'] = [analyzer.polarity_scores(x)['neu'] for x in wall['body_split']]
#wall['pos'] = [analyzer.polarity_scores(x)['pos'] for x in wall['body_split']]


wall.head()


C:\dev\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,title,score,id,url,comms_num,created,body,timestamp,title_split,body_split,title_nostop,body_nostop,compound,neg,neu,pos,rating
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"[Exit, the, system]","[The, CEO, of, NASDAQ, pushed, to, halt, tradi...","[Exit, system]","[The, CEO, NASDAQ, pushed, halt, trading, “to,...",0.0,0.0,1.0,0.0,"{'neg': 0.07, 'neu': 0.852, 'pos': 0.078, 'com..."
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,"[SHORT, STOCK, DOESN'T, HAVE, AN, EXPIRATION, ...","[Hedgefund, whales, are, spreading, disinfo, s...","[SHORT, STOCK, DOESN'T, HAVE, AN, EXPIRATION, ...","[Hedgefund, whales, spreading, disinfo, saying...",0.0,0.0,1.0,0.0,"{'neg': 0.053, 'neu': 0.835, 'pos': 0.112, 'co..."
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,"[THIS, IS, THE, MOMENT]","[Life, isn't, fair., My, mother, always, told,...","[THIS, IS, THE, MOMENT]","[Life, fair., My, mother, always, told, I, wou...",0.0,0.0,1.0,0.0,"{'neg': 0.177, 'neu': 0.768, 'pos': 0.056, 'co..."
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,"[We, need, to, keep, this, movement, going,, w...","[I, believe, right, now, is, one, of, those, r...","[We, need, keep, movement, going,, make, histo...","[I, believe, right, one, rare, opportunities, ...",0.0,0.0,1.0,0.0,"{'neg': 0.093, 'neu': 0.735, 'pos': 0.172, 'co..."
12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,"[Once, you're, done, with, GME, -, $AG, and, $...","[You, guys, are, champs., GME..., who, would, ...","[Once, done, GME, -, $AG, $SLV,, gentleman's, ...","[You, guys, champs., GME..., would, thought, b...",0.0,0.0,1.0,0.0,"{'neg': 0.068, 'neu': 0.775, 'pos': 0.156, 'co..."


In [83]:
import nltk 
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Allegra
[nltk_data]     Grunberg\AppData\Roaming\nltk_data...


True

In [84]:
#sentiment analysis take 2 
#try this with body
#same as vader! 

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sid = SIA()
wall['sentiments']           = wall['body_nostop'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
wall['Positive Sentiment']   = wall['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
wall['Neutral Sentiment']    = wall['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
wall['Negative Sentiment']   = wall['sentiments'].apply(lambda x: x['neg']+1*(10**-6))

AttributeError: 'list' object has no attribute 'lower'

# 3. Converting Emoji's to meanings

- https://www.kaggle.com/iamprateek/emoji-analysis
- https://www.geeksforgeeks.org/convert-emoji-into-text-in-python/

In [88]:
use_cols=['Char','Neg','Neut','Pos']
train_df=pd.read_csv('reddit_wsb.csv',usecols=use_cols)
train_df.head(20)

#wall = pd.read_csv('reddit_wsb.csv') 

ValueError: Usecols do not match columns, columns expected but not found: ['Pos', 'Neut', 'Neg', 'Char']

In [ ]:
#drop the first row
train_df.drop(train_df.index[0], inplace=True)
train_df.head()

In [ ]:
#reset the index
train_df = train_df.reset_index(drop=True)
train_df.head()

In [ ]:
#change into pandas category
train_df["Char"] = train_df["Char"].astype('category')
train_df["Neg"] = train_df["Neg"].astype('float')
train_df["Neut"] = train_df["Neut"].astype('float')
train_df["Pos"] = train_df["Pos"].astype('float')

In [ ]:
train_df.info()

In [ ]:
train_df["Char_cat"] = train_df["Char"].cat.codes
train_df.head()

In [ ]:
dataset=train_df.drop(['Char'],axis=1)
dataset.head()

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
arrays = dataset.values
imp.fit(arrays)
array_imp = imp.transform(arrays)

In [ ]:
array_imp.shape

In [ ]:
X=array_imp[:,-1]
print(X.shape)
X=X.reshape(-1,1)
Y=array_imp[:,3]
print(Y.shape)

In [ ]:
validation_size = 0.20
seed = 7

In [ ]:
#modeling 

from sklearn import model_selection
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=7)

In [ ]:
svc = LogisticRegression()
svc.fit(X_train, Y_train)
predictions = svc.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
#let's test
x = ['👍']
arr=np.array(x)
x_encoded=arr.reshape(-1,1)
#x_encoded = x.cat.codes
svc.predict(x_encoded)

Saving Output

In [ ]:
wall_NLP.to_csv('output.csv')